In [44]:
from os import chdir
from itertools import compress
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, col, lit, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, CountVectorizer
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
import json
import re
print("spark ran")

chdir("/home/centos/Desktop/ingest")

spark ran


In [45]:
#Creation of filtering words
with open("/home/centos/Desktop/ingest/Overall_Master.txt", encoding="ISO-8859-1") as file:
    ingredient_set = file.read().splitlines()

print("List length: " +str(len(ingredient_set)))
print(ingredient_set)
ingredient_set = ingredient_set[1:10]

List length: 524
['longan', 'calalu', 'mutton', 'langoustine', 'paprika', 'plaice', 'coriander', 'marron', 'basil', 'noisette', 'buttermilk', 'cranberry', 'liverwurst', 'tansy', 'veal', 'miso', 'shaddock', 'cockle', 'cashew', 'mousetrap', 'cumin', 'round', 'chicory', 'pineapple', 'margarine', 'cherry', 'ham', 'dolcelatte', 'lamb', 'durion', 'grapefruit', 'game', 'bass', 'butter', 'tayberry', 'skirt', 'haslet', 'choko', 'halibut', 'pomegranate', 'jarlsberg', 'cardamom', 'courgette', 'chervil', 'mackerel', 'forehock', 'sardine', 'taleggio', 'saffron', 'bakeapple', 'shoulder', 'greens', 'queenie', 'gorgonzola', 'salmon', 'lemon', 'anchovy', 'prune', 'pistachio', 'chayote', 'terakihi', 'nut', 'teraglin', 'cervelat', 'pimiento', 'clove', 'snook', 'sapota', 'gooseberry', 'bonito', 'clappy-doo', 'cutlet', 'squid', 'chop', 'pollack', 'octopus', 'bramble', 'pike', 'cumquat', 'kahawai', 'chard', 'yam', 'broccoli', 'romano', 'pomelo', 'brill', 'sprat', 'huss', 'chestnut', 'edam', 'apple', 'tamari

In [46]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
print("Spark Session Created")
#Python Java issues: a drawback in using pyspark instead of Scala

Spark Session Created


In [47]:
path = "/home/centos/Desktop/ingest/allrecipes-recipes.json"
allrecipes_df = spark.read.json(path)
allrecipes_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- cook_time_minutes: long (nullable = true)
 |-- description: string (nullable = true)
 |-- error: boolean (nullable = true)
 |-- footnotes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- photo_url: string (nullable = true)
 |-- prep_time_minutes: long (nullable = true)
 |-- rating_stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- time_scraped: long (nullable = true)
 |-- title: string (nullable = true)
 |-- total_time_minutes: long (nullable = true)
 |-- url: string (nullable = true)



In [48]:
#create temporary view to transform to dataframes
allrecipes_df.createOrReplaceTempView("allrecipes")

#overall
allrecipes_df = spark.sql("SELECT title, description, ingredients, instructions, rating_stars, review_count, photo_url, total_time_minutes, url FROM allrecipes LIMIT 1000")
allrecipes_df.printSchema()
allrecipes_df.show(5)

#to generate into sample
# small_sample = allrecipes_df.toPandas()
# small_sample.to_csv("small_sample.csv")

#Noted cook_time_minutes have quite a bit of zeroes. Use rating_stars and review_count instead


root
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- instructions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rating_stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- photo_url: string (nullable = true)
 |-- total_time_minutes: long (nullable = true)
 |-- url: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+------------------+--------------------+
|               title|         description|         ingredients|        instructions|rating_stars|review_count|           photo_url|total_time_minutes|                 url|
+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+------------------+--------------------+
|Bas

In [49]:
#User-defined Functions
def removeParenthesis(li):
    output = []
    for text in li:
        text = re.sub(r"\([^)]*\)", "", text)
        text = text.strip()
        output.append(text)
    return output

def extractIngredient(li, ingredient_set):
    output = []
    for item in li:
        temp_output = []
        for set_tracker in range(len(ingredient_set)):
            check = bool(re.search(ingredient_set[set_tracker], item))
            if (check == True):    
                temp_output.append(ingredient_set[set_tracker])
        if (len(temp_output) != 0):
#Comment
#             temp_counter = 0
#             temp_tracker = 0
#             for temp in range(len(temp_output)):
#                 count_temp = len(temp_output[temp])
#                 if (count_temp > temp_counter):
#                     temp_tracker = temp
#                     temp_counter = count_temp
#             output.append(temp_output[temp_tracker])
#Comment
            output.append(temp_output[[len(i) for i in temp_output].index(max([len(i) for i in temp_output]))])
    return output

sample = ['1/2 cup unsalted butter, chilled and cubed', '1 cup chopped onion', '1 3/4 cups cornmeal', '1 1/4 cups all-purpose flour', '1/4 cup white sugar', '1 tablespoon baking powder', '1 1/2 teaspoons salt', '1/2 teaspoon baking soda', '1 1/2 cups buttermilk', '3 eggs', '1 1/2 cups shredded pepperjack cheese', '1 1/3 cups frozen corn kernels, thawed and drained', '2 ounces roasted marinated red bell peppers, drained and chopped', '1/2 cup chopped fresh basil']

test = extractIngredient(sample, ingredient_set)
print(test)

#create user defined function
udf_removeParen = udf(removeParenthesis, ArrayType(StringType()))
udf_extractIngredient = udf(lambda x: extractIngredient(x,ingredient_set), ArrayType(StringType()))
udf_countList = udf(lambda x: len(x), IntegerType())

print("UDF Successful.")

['basil']
UDF Successful.


In [50]:
#create ingredient table
ingredient_df = allrecipes_df.select(["title", "ingredients"])
cleanIngredient_df = ingredient_df.withColumn('rm_paren', udf_removeParen('ingredients')).select('title','ingredients','rm_paren') \
                         .withColumn('ingredient_extract', udf_extractIngredient('rm_paren')).select('title','ingredient_extract') \
                         .withColumn('extract_count', udf_countList('ingredient_extract')).select('title','ingredient_extract', 'extract_count')
cleanIngredient_df.show(5)

#to convert into graph form
ingredient_graph = cleanIngredient_df.withColumn('exploded',explode('ingredient_extract')) \
                      .select(col('title').alias('Recipe'),col('exploded').alias('Ingredient'))
#                       .withColumn("Frequency", lit(1))
ingredient_graph = ingredient_graph.filter(ingredient_graph.Ingredient != "")
ingredient_graph.show(20)

# graph_sample = ingredient_graph.toPandas()
# graph_sample.to_csv("graph_sample.csv")

#Let's try convert ingredient into count_frequency (BAD WAY, don't do)
# ingredient_freq = ingredient_graph.groupBy("Recipe").pivot("Ingredient").sum("Frequency")
# ingredient_freq.show()
# test = ingredient_freq.toPandas()
# test.to_csv("test.csv")
# print("test done")


#Proof of concept (Need to do !=)
# print(ingredient_graph.filter(ingredient_graph.Ingredient.isNull()).count())
# ingredient_graph = ingredient_graph.na.drop(subset=["Recipe"])
# test = ingredient_graph.filter(ingredient_graph.Recipe == "Grandma Emma's Spice Loaf")
# test.show()
# test2 = test.filter(ingredient_graph.Ingredient != "")
# test2.show()


+--------------------+------------------+-------------+
|               title|ingredient_extract|extract_count|
+--------------------+------------------+-------------+
|Basil, Roasted Pe...|           [basil]|            1|
|Crispy Cheese Twists|                []|            0|
|   Mom's Yeast Rolls|                []|            0|
|Sweet Potato Bread I|                []|            0|
|         Orange Buns|                []|            0|
+--------------------+------------------+-------------+
only showing top 5 rows

+--------------------+----------+
|              Recipe|Ingredient|
+--------------------+----------+
|Basil, Roasted Pe...|     basil|
|Sun Dried Tomato ...|     basil|
|Italian Herb Bread I|     basil|
|     Cheddar Muffins|   paprika|
|Whole Wheat Zucch...|     basil|
|        Garlic Bread|     basil|
|Garlic and Herb B...|     basil|
|Tomato Basil Sher...|     basil|
|            Fougasse|     basil|
|   Cheese Herb Bread|     basil|
|   Marron Layer Cake|    mar

In [52]:
#User Defined Function - Feature Engineering 
#Labels: Vegetarian, Lactose, Nut, Seafood

#Safer to start off as non-vegetarian
def detectVege(li, vegDetect_list):
    label = 0
    detect_list = []
    for text in li:
        if text in vegDetect_list:
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 1
    return label

#Safer to start off as positive allergy
def detectNut(li):
    label = 1
    detect_list = []
    for text in li:
        if ("nut" in text):
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 0
    return label

def detectDairy(li):
    label = 1
    dairy_list = ["cheese", "milk", "yoghurt", "cream"]
    detect_list = []
    for text in li:
        if (text in dairy_list):
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 0
    return label

def detectSeafood(li, seaDetect_list):
    label = 1
    detect_list = []
    for text in li:
        if text in seaDetect_list:
            detect_list.append(text)
    if (len(detect_list) == 0):
        label = 0
    return label


with open("vegDetect.txt", encoding="ISO-8859-1") as veg_file:
    vegDetect_list = veg_file.read().splitlines()

with open("seafoodDetect.txt", encoding="ISO-8859-1") as seafood_file:
    seaDetect_list = seafood_file.read().splitlines()
    
#create user defined function
udf_detectVege = udf(lambda x: detectVege(x, vegDetect_list), IntegerType())
udf_detectNut = udf(detectNut, IntegerType())
udf_detectDairy = udf(detectDairy, IntegerType())
udf_detectSeafood = udf(lambda x: detectSeafood(x, seaDetect_list), IntegerType())

#experimentation of labelling
labelling_df = cleanIngredient_df.select("title", "ingredient_extract")
labelling_df = labelling_df.withColumn("vegetarian_label", udf_detectVege("ingredient_extract")) \
                           .withColumn("nut_label", udf_detectNut("ingredient_extract")) \
                           .withColumn("lactose", udf_detectDairy("ingredient_extract")) \
                           .withColumn("seafood", udf_detectSeafood("ingredient_extract"))

labelling_df.show(5)


+--------------------+------------------+----------------+---------+-------+-------+
|               title|ingredient_extract|vegetarian_label|nut_label|lactose|seafood|
+--------------------+------------------+----------------+---------+-------+-------+
|Basil, Roasted Pe...|           [basil]|               1|        0|      0|      0|
|Crispy Cheese Twists|                []|               1|        0|      0|      0|
|   Mom's Yeast Rolls|                []|               1|        0|      0|      0|
|Sweet Potato Bread I|                []|               1|        0|      0|      0|
|         Orange Buns|                []|               1|        0|      0|      0|
+--------------------+------------------+----------------+---------+-------+-------+
only showing top 5 rows



In [56]:
# test = labelling_df.toPandas()
# test.head()
test.to_csv("test.csv")
# test = test.to_csv("test.csv")

In [ ]:
cleanIngredient_table = cleanIngredient_df.select("title", "rm_complete")
cleanIngredient_table = cleanIngredient_table.withColumn("Ing_Str", udf_convertString("rm_complete"))
# cleanIngredient_table = cleanIngredient_table.withColumn("Ing_Str", udf_convertString2("rm_complete"))
cleanIngredient_table = cleanIngredient_table.select("title", col('Ing_Str').alias('Ingredient'))
cleanIngredient_table = cleanIngredient_table.withColumn("ID", monotonically_increasing_id())

cleanIngredient_table.show(5)
cleanIngredient_table.count()

cleanIngredient_pandas = cleanIngredient_table.toPandas()
cleanIngredient_pandas.to_csv("sample_clean.csv")

# cleanIngredient_table.createOrReplaceTempView('df')
# cleanIngredient_table = spark.sql('select row_number() over (order by "ID") as num, * from df')
# cleanIngredient_table.show(5)
# cleanIngredient_table.count()


In [ ]:
sim_mat = IndexedRowMatrix(sim_test).toBlockMatrix()
dot = sim_mat.multiply(sim_mat.transpose())
dot.toLocalMatrix().toArray()
dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())

similarity.alias("i").join(similarity.alias("j"), col("i.ID") < col("j.ID"))\
    .select(
        col("i.ID").alias("i"), 
        col("j.ID").alias("j"), 
        dot_udf("i.ID", "j.ID").alias("cosine"))\
    .sort("i", "j")\
    .show()
